# Exercise - Regression


The data set for this exercise includes information on house sales in King County, WA (between May 2014 and May 2015). (Each row in the data set pertains to one house. There is a total of 21,613 houses in the data set). Use this data set to predict the sale price of a house (i.e., the `price` column) based on the characteristics of the house. A model can be helpful for buyers, sellers, realtors, and lenders.

## Description of Variables

The description and type of each variable is provided in "KC house data - Data Dictionary.docx". Make sure to read this document to learn about the variables.

## Goal

Use the **kc_house_data.csv** data set and build a model to predict **price**. <br>

# Read and Prepare the Data

In [8]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(99)

In [9]:
#We will predict the "price" value in the data set:

housing = pd.read_csv("kc_house_data (1).csv")
housing.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,432000.0,5.0,2.75,2060.0,329903.0,1.5,0,3,5,7.0,2060,0,1989.0,0,98022.0,47.1776,-121.944,2240,220232.0
1,170000.0,2.0,1.00,810.0,8424.0,1.0,0,0,4,6.0,810,0,1959.0,0,98023.0,47.3286,-122.346,820,8424.0
2,235000.0,3.0,1.00,960.0,5030.0,1.0,0,0,3,7.0,960,0,1955.0,0,98118.0,47.5611,-122.280,1460,5400.0
3,350000.0,2.0,1.00,830.0,5100.0,1.0,0,0,4,7.0,830,0,1942.0,0,98126.0,47.5259,-122.379,1220,5100.0
4,397380.0,2.0,1.00,1030.0,5072.0,1.0,0,0,3,6.0,1030,0,1924.0,1958,98115.0,47.6962,-122.294,1220,6781.0


# Split data (train/test)

In [58]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(housing, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [60]:
train_target = train['price']
test_target = test['price']

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)
train_target

11646     167000.0
1716      225900.0
6893      750000.0
7041      494000.0
14291     329900.0
           ...    
2487      315000.0
1349      550000.0
14969     390000.0
5612     1000000.0
10290     325000.0
Name: price, Length: 15129, dtype: float64

## Set the data type of "zipcode" to categorical

In [61]:
train_inputs['zipcode'] = train_inputs['zipcode'].astype('object')
test_inputs['zipcode'] = test_inputs['zipcode'].astype('object')

##  Identify the numeric, binary, and categorical columns

In [62]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [63]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['waterfront']

In [64]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [65]:
binary_columns

['waterfront']

In [66]:
numeric_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [67]:
categorical_columns

['zipcode']

# Pipeline

In [68]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [69]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [70]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [71]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

In [72]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [73]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<15129x88 sparse matrix of type '<class 'numpy.float64'>'
	with 257307 stored elements in Compressed Sparse Row format>

In [74]:
train_x.shape

(15129, 88)

# Tranform: transform() for TEST

In [75]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<6484x88 sparse matrix of type '<class 'numpy.float64'>'
	with 110277 stored elements in Compressed Sparse Row format>

In [76]:
test_x.shape

(6484, 88)

# Calculate the Baseline

In [77]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(train_x, train_target)

DummyRegressor()

In [78]:
from sklearn.metrics import mean_squared_error

In [79]:
# This is the baseline Train RMSE

dummy_train_pred = dummy_regr.predict(train_x)

baseline_train_mse = mean_squared_error(train_target, dummy_train_pred)

baseline_train_rmse = np.sqrt(baseline_train_mse)

print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

Baseline Train RMSE: 368533.6918352049


In [80]:
# This is the baseline Test RMSE

dummy_test_pred = dummy_regr.predict(test_x)

baseline_test_mse = mean_squared_error (test_target, dummy_test_pred)

baseline_test_rmse = np.sqrt(baseline_test_mse)

print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

Baseline Test RMSE: 364590.4181375267


# Train a SGD model (with no regularization)

In [81]:

from sklearn.linear_model import SGDRegressor 

sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.01) 

sgd_reg.fit(train_x, train_target)


SGDRegressor(max_iter=100, penalty=None)

In [82]:
sgd_reg.predict(test_x)

array([149014.79767932, 375062.34704398, 408036.89785401, ...,
       817136.6909433 , 449537.84607418, 344528.77840222])

In [83]:
predictions = pd.DataFrame(sgd_reg.predict(test_x), columns=['Predicted'])

predictions

,Predicted
0,1.490148e+05
1,3.750623e+05
2,4.080369e+05
3,4.785226e+05
4,2.795869e+05
...,...
6479,1.003250e+06
6480,2.367403e+05
6481,8.171367e+05
6482,4.495378e+05


In [84]:
predictions['Actual'] = np.array(test_target)

predictions

,Predicted,Actual
0,1.490148e+05,230000.0
1,3.750623e+05,345000.0
2,4.080369e+05,485000.0
3,4.785226e+05,386000.0
4,2.795869e+05,220000.0
...,...,...
6479,1.003250e+06,910000.0
6480,2.367403e+05,250600.0
6481,8.171367e+05,575000.0
6482,4.495378e+05,428000.0


### Generate the error metrics

In [85]:
#Train RMSE
reg_train_pred = sgd_reg.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE value is: {}' .format(train_rmse))


Train RMSE value is: 164920.41160927515


In [86]:
#Test RMSE
reg_test_pred = sgd_reg.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE value is: {}' .format(test_rmse))


Test RMSE value is: 160887.32726262094


# Try L1 Regularization in SGD

In [87]:
#Stochastic Gradient:
sgd_reg_L1 = SGDRegressor(max_iter=50, penalty='l1', alpha = 1, eta0=0.01)

sgd_reg_L1.fit(train_x, train_target)

C:\Users\17272\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(alpha=1, max_iter=50, penalty='l1')

### Generate the error metrics

In [88]:
#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))


Train RMSE: 166483.6634491571


In [89]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 162576.28364163658


# Try L2 Regularization in SGD

In [90]:
#Stochastic Gradient:

sgd_reg_L2 = SGDRegressor(max_iter=50, penalty='l2', alpha = 0.1, eta0=0.01)


In [91]:

sgd_reg_L2.fit(train_x, train_target)


C:\Users\17272\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(alpha=0.1, max_iter=50)

### Generate the error metrics

In [92]:
#Train RMSE
reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 205496.76861274


In [93]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 201725.80521829036


# Try ElasticNet in SGD

In [94]:
#Stochastic Gradient:
sgd_reg_elastic = SGDRegressor(max_iter=50, penalty='elasticnet', l1_ratio=0.5, alpha = 0.1, 
                          eta0=0.01)
sgd_reg_elastic.fit(train_x, train_target)


C:\Users\17272\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(alpha=0.1, l1_ratio=0.5, max_iter=50, penalty='elasticnet')

### Generate the error metrics

In [95]:
#Train RMSE
reg_train_pred = sgd_reg_elastic.predict(train_x)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 206511.61413440693


In [96]:
#Test RMSE
reg_test_pred = sgd_reg_elastic.predict(test_x)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 202255.6100097896


# Create Polynomial Features

Create polynomial features with degree = 2. 

In [97]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms and interaction terms
poly_features = PolynomialFeatures(degree=2).fit(train_x)

train_x_poly = poly_features.transform(train_x)

test_x_poly = poly_features.transform(test_x)


In [98]:
pol_lin_reg = SGDRegressor(max_iter=1000, penalty=None, eta0=0.01) 

pol_lin_reg.fit(train_x_poly, train_target)

SGDRegressor(penalty=None)

# Try L2 Regularization in SGD (with polynomial features)

In [99]:
pol_lin_reg_l2 = SGDRegressor(max_iter=50, penalty='l2', alpha = 0.1, eta0=0.01) 

pol_lin_reg_l2.fit(train_x_poly, train_target)


SGDRegressor(alpha=0.1, max_iter=50)

### Generate the error metrics

In [100]:
#Train RMSE
reg_train_pred = pol_lin_reg.predict(train_x_poly)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 9489873212327.719


In [101]:
#Test RMSE
reg_test_pred = pol_lin_reg.predict(test_x_poly)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 10724475732197.975


# Regularization using ElasticNet

In [102]:
elastic_net = SGDRegressor(max_iter=1000, penalty='elasticnet', l1_ratio=0.5, alpha = 100, 
                          eta0=0.01)

elastic_net.fit(train_x_poly, train_target)

SGDRegressor(alpha=100, l1_ratio=0.5, penalty='elasticnet')

In [103]:
#Train RMSE
reg_train_pred = elastic_net.predict(train_x_poly)

train_mse = mean_squared_error(train_target, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 385759.7261770971


In [104]:
#Test RMSE
reg_test_pred = elastic_net.predict(test_x_poly)

test_mse = mean_squared_error (test_target, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 384699.5762795905
